# Curbing Chicago Car Crashes

**Authors:** Carlos McCrum, Michael Lee, Doug Mill
***

## Overview

Our task is to build inferential classification models for the Vehicle Safety Board of Chicago. We cleaned and formatted our data provided by the City of Chicago containing crashes,  vehicles and people relating to crashes from 2016 to 2020, we then modeled the primary contributory causes of car accidents into two categories. We used an iterative modeling approach and incorporated several classification models to see if we could find what crashes were preventable. Our recommendations include investing in driver education for certain age groups and fixing certain road conditions that could cause a crash. 
***

## Business Problem

Vehicle Safety Board wants to know if they should spend more funding in drivers education or fixing roads. Determining whether a crash is preventable and how to allocate funds to limit the number of crashes to present to the Vehicle Safety Board of Chicago.


***

## Data Understanding

Our stakeholder is the Vehicle Safety Board of Chicago. They are launching a new campaign to reduce car crashes. Our task is to build an inferential model to find out  which crashes were preventable and not. We labeled ‘Preventable’ as crashes that could have easily been avoided. Not following traffic laws and negligent driving would fall under this category. ‘Less Preventable’ are crashes that would require a substantial amount of money, time, and labor to fix. Bad road conditions, vision obscurity, and bad weather conditions would fall under this category.
***

##### Start by importing necessary packages. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import plot_confusion_matrix, roc_auc_score, plot_roc_curve
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

***

## Data Loading

Import data previously cleaned with only the relevant columns we need for our analysis. Our variable is a binary target that points to the contributory cause of the crash either being preventable or non-preventable.

***


Import our cleaned dataframe we cleaned previously. 

In [2]:
main_df = pd.read_csv('data/cleaned_data/cleaned_modeling_data.csv')
# Remove Bad Weather as it was too highly correlated with Bad Road Conditions
columns_to_drop = ['Unnamed: 0', 'BAD_WEATHER']
main_df.drop((columns_to_drop), axis = 1, inplace = True)

Check to see if we have a class imbalance. 

In [3]:
main_df['Target'].value_counts(normalize = True)

0    0.754374
1    0.245626
Name: Target, dtype: float64

Check correlations with our target

In [4]:
# sns.set(rc={'figure.figsize':(20, 20)})
# sns.heatmap(main_df.corr().abs(),annot=True);


## Data Modeling
We modeled the data through iterative modeling. We used a logistic regression model as our first simple model. For our second model, we created a Decision Tree Classifier that scored slightly better than our simple model. We used a RFE to determine the most important features and iterated with GridSearch to find the best parameters. Lastly, we used a XGBoost classifier with GridSearchCV to find the best model.
***

## Simple Model - Logistic Regression

To run our first simple model we start off by dropping irrelevant columns. 

In [5]:
columns_to_drop = ['CRASH_RECORD_ID','CRASH_DATE', 'LONGITUDE', 'LATITUDE', 'Target']

Split our data with train test split 

In [6]:
X = main_df.drop((columns_to_drop), axis=1)
y = main_df['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [7]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
logreg.score(X_test, y_test)

0.7466465015648959

In [9]:
print(logreg.coef_, logreg.intercept_)

[[ 0.12988649  0.20098016  0.822363    1.42466443 -0.44294368 -0.07022554
   0.45626927 -0.40778448  0.90737109 -2.07794228]] [-0.69633504]


In [12]:
from statsmodels.discrete.discrete_model import Logit
from statsmodels.tools import add_constant
X = add_constant(X)
print(Logit(y, X).fit().summary())

Optimization terminated successfully.
         Current function value: 0.481552
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                 Target   No. Observations:               952140
Model:                          Logit   Df Residuals:                   952129
Method:                           MLE   Df Model:                           10
Date:                Wed, 27 Oct 2021   Pseudo R-squ.:                  0.1362
Time:                        13:25:40   Log-Likelihood:            -4.5851e+05
converged:                       True   LL-Null:                   -5.3080e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     -0.6939      0.005   -143.200      0.000      -0.703      

Because our class balance is 3:1, we are not going to SMOTE the minority class.

## Using RFE to select the most important columns

In [ ]:
dt = DecisionTreeClassifier()
num_features_to_select = 5
use_RSE = True

select = RFE(dt, n_features_to_select=num_features_to_select)
select.fit(X = X_train, y = y_train)

keep_list = [(k,v) for k,v in zip(main_df.drop((columns_to_drop), axis=1).columns,select.support_)]
keep_string = ''
for k,v in keep_list:
    if v:
        keep_string += k + ','
if use_RSE == False:
    keep_list = formula_string.split('+')
keep_string = keep_string[:-1]

keep_string

In [ ]:
X = main_df[['BAD_ROAD_CONDITION','DEFECT_ROAD','WORKING_TRAFFIC_DEVICE','OBSCURED_VIZ','DRIVER_ERROR']]
y = main_df['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

dt_rfe = DecisionTreeClassifier(random_state = 42)
dt_rfe.fit(X_train, y_train)
dt_rfe.score(X_test, y_test)

In [ ]:
X = main_df.drop((columns_to_drop), axis = 1)
y = main_df['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

dt = DecisionTreeClassifier(random_state = 42)
dt.fit(X_train, y_train)
dt.score(X_test, y_test)

In [ ]:
sns.set(rc={'figure.figsize':(10, 10)})
plot_confusion_matrix(dt, X_train, y_train)
plt.grid('OFF');

In [ ]:
plot_confusion_matrix(dt, X_test, y_test);

# Grid Search

Because Decision Trees gave us the best accuracy score, we decided to try and find the best parameters
Using Grid Search to find the best parameters

In [ ]:
# Look through max depth, min samples split, min sample leaf to find the best perameters. 
grid = [{'max_depth': [4, 6, 8], 
         'min_samples_split': [5, 10, 15], 
         'min_samples_leaf': [20, 25, 30]}]
GS = GridSearchCV(estimator=dt, 
                          param_grid=grid, 
                          scoring='recall', 
                          cv=5)

Fit to our grid

In [ ]:
GS.fit(X_train, y_train)

- Analyze the results

In [ ]:
GS.cv_results_

- Find our best score

In [ ]:
GS.best_estimator_.score(X_test, y_test)

- See what the best perameters are

In [ ]:
GS.best_params_

## XGBoost

- Initiate XGBClassifier, fit, and determine the training and validation scores. 

In [ ]:
clf = XGBClassifier(eval_metric='mlogloss')
clf.fit(X_train, y_train)
training_preds = clf.predict(X_train)
test_preds = clf.predict(X_test)

# Accuracy
training_accuracy = accuracy_score(y_train, training_preds)
test_accuracy = accuracy_score(y_test, test_preds)
print('Training Accuracy: {:.4}%'.format(training_accuracy * 100))
print('Validation accuracy: {:.4}%'.format(test_accuracy * 100))
print('Test Accuracy: {}')

In [ ]:
clf.score(X_test, y_test)

In [ ]:
param_grid = {
    'learning_rate': [0.2, 0.5],
    'max_depth': [6],
    'min_child_weight': [1],
    'subsample': [0.7],
    'n_estimators': [100, 200],
}

In [ ]:

grid_clf = GridSearchCV(clf, param_grid, scoring='accuracy', cv=None, n_jobs=1)
grid_clf.fit(X_train, y_train)

best_parameters = grid_clf.best_params_

print('Grid Search found the following optimal parameters: ')
for param_name in sorted(best_parameters.keys()):
    print('%s: %r' % (param_name, best_parameters[param_name]))

training_preds = grid_clf.predict(X_train)
test_preds = grid_clf.predict(X_test)
training_accuracy = accuracy_score(y_train, training_preds)
test_accuracy = accuracy_score(y_test, test_preds)

print('')
print('Training Accuracy: {:.4}%'.format(training_accuracy * 100))
print('Validation accuracy: {:.4}%'.format(test_accuracy * 100))

In [ ]:
grid_clf.score(X_test, y_test)

## Results

The results of our model indicated that most of the crashes were Preventable. By spending more money on drivers education in ages 20 - 39, we could curb the total accidents in Chicago drastically.

## Evaluation
Evaluate how well your work solves the stated business problem.

***
Questions to consider:
* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?
***

## Conclusions

We recommend investing in online drivers education because this could significantly lower the amount of preventable crashes in the Chicago area. We recommend driver education because it is affordable, easy to implement, and saves a lot more in labor costs vs. repairing all the roads in Chicago.

***
Questions to consider:
* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?
***